In [ ]:
import numpy as np
import pandas as pd
import random
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options # type: ignore
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# Cấu hình trình duyệt
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Giảm phát hiện bot
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Khởi tạo trình duyệt với WebDriver Manager (tự động tải đúng ChromeDriver)
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Mở URL
url = "http://tripadvisor.com/Restaurants-g298085-Da_Nang.html"
driver.get(url)
time.sleep(15)  # Chờ 15 giây để đảm bảo trang tải

In [2]:
# Lấy title và links
elems = driver.find_element(By.CSS_SELECTOR , "div[data-automation='searchResults']")
spans = elems.find_elements(By.XPATH, './span')
for span in spans:
    link = span.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
    print(link)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-automation='searchResults']"}
  (Session info: chrome=140.0.7339.208); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x84c333+65459]
	GetHandleVerifier [0x0x84c374+65524]
	(No symbol) [0x0x66d973]
	(No symbol) [0x0x6b76e7]
	(No symbol) [0x0x6b7a8b]
	(No symbol) [0x0x6fdea2]
	(No symbol) [0x0x6d9e44]
	(No symbol) [0x0x6fb606]
	(No symbol) [0x0x6d9bf6]
	(No symbol) [0x0x6ab38e]
	(No symbol) [0x0x6ac274]
	GetHandleVerifier [0x0xaceda3+2697763]
	GetHandleVerifier [0x0xac9ec7+2677575]
	GetHandleVerifier [0x0x874194+228884]
	GetHandleVerifier [0x0x8649f8+165496]
	GetHandleVerifier [0x0x86b18d+192013]
	GetHandleVerifier [0x0x8547d8+99416]
	GetHandleVerifier [0x0x854972+99826]
	GetHandleVerifier [0x0x83ebea+10346]
	BaseThreadInitThunk [0x0x76a95d

In [ ]:
for span in spans:
    xpaths = [
        './div/div/div/div[2]/div[1]/div/div[2]/div[1]/div[2]/a',
        './div/div/div[2]/div[1]/div/div[2]/div[1]/div/a'
    ]
    tilte = None
    for xpath in xpaths:
        try:
            tilte = span.find_element(By.XPATH, xpath)
            break  # Nếu tìm thấy thì dừng lại
        except:
            continue  # Nếu lỗi thì thử XPath tiếp theo
    if tilte:
        print(tilte.text)
    else:
        print("Không tìm thấy tiêu đề cho một span")


In [ ]:
# Danh sách các XPath có thể xảy ra
xpaths = [
    "./div/div/div/div[2]/div[1]/div/div[2]/div[2]/div/div[1]/span[1]/span/span/a/span[2]",
    "./div/div/div[2]/div[1]/div/div[2]/div[2]/div[1]/div[1]/span[1]/span/span/a/span[2]"
]
for span in spans:
    for xpath in xpaths:
        try:
            target_span = span.find_element(By.XPATH, xpath)
            Review = target_span.text.strip()
            if Review:  # Nếu tìm thấy text thì in ra và dừng vòng lặp
                print(Review)
                break
        except:
            pass  # Nếu không tìm thấy phần tử thì thử XPath tiếp theo


In [ ]:
# Lấy title
elems = driver.find_elements(By.CSS_SELECTOR , ".ABufV")
title = [elem.text for elem in elems]
print(title)

In [ ]:
len(title)

In [ ]:
# Tìm tất cả phần tử chứa rating & reviews
elems_rating = driver.find_elements(By.CSS_SELECTOR, "div.jVDab.W.f.u.w.JqMhy")

# Lấy danh sách nội dung từ thuộc tính aria-label
ratings = [elem.get_attribute("aria-label") for elem in elems_rating]

print("Danh sách Rating & Reviews:", ratings)

In [ ]:
len(ratings)

In [ ]:
# Lấy num_of_reviews
elems_numreviews = driver.find_elements(By.CSS_SELECTOR , ".yyzcQ")
num_of_reviews = [elem.text for elem in elems_numreviews]
print(num_of_reviews)

In [ ]:
len(num_of_reviews)

In [ ]:
# Lấy Cuisines
elems_cuisines = driver.find_elements(By.CSS_SELECTOR , ".HWdbN.o.W.q")
cuisines = [elem.text for elem in elems_cuisines if elem.text and "reviews" not in elem.text]
print(cuisines)

In [ ]:
len(cuisines)

In [ ]:
# Lấy price
elems_price = driver.find_elements(By.CSS_SELECTOR , ".BUNnJ")
price = [elem.text for elem in elems_price if not elem.text or "$" in elem.text]
print(price)

In [ ]:
len(price)

In [ ]:
import pandas as pd

# Tạo DataFrame từ các danh sách đã thu thập
data = {
    "Title": title,
    "Links": links,
    "Ratings": ratings,
    "Num_of_Reviews": num_of_reviews,
    "Cuisines": cuisines,
    "Price": price
}

# Chuyển đổi thành DataFrame, tự động xử lý độ dài khác nhau bằng cách điền NaN
df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))

# Hiển thị DataFrame
print(df)

In [ ]:
df.to_csv("restaurant_data.csv", index=False, encoding="utf-8-sig")